<h1 align="center"> FYP Live Report </h1>


## Imports and stuff

The library titled py_dss_interface comes with its own version of openDSS and so, we do not need to install openDSS separately.

In [1]:
import py_dss_interface
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import math

Next, we set the path of the dss file. I would prefer to use the python OS module so that writing the path is never an issue again, but unfortunately in live scripts such as these, we are prone to 

In [12]:
dss_file = "..\\13Bus\IEEE13Nodeckt.dss"

print(dss_file)

..\13Bus\IEEE13Nodeckt.dss


Creating a variable of the self defined data type as shown

In [9]:
dss = py_dss_interface.DSSDLL()

OpenDSS Started successfully! 
OpenDSS Version 9.4.0.1 (64-bit build); License Status: Open 




Compiling the script, which is automatically executed due to "solve" command in the code

In [11]:
dss.text(f"compile [{dss_file}]")

''

Next, we try to get the voltages, powers, and currents at every node in the circuit. Next, we will attempt to do a timeseries simulation of the circuit where we will sweep one load's values over some duration an examine the effects on the parameters of other components.


In [15]:
loads = [
"Load.671",
"Load.634a",
"Load.634b",
"Load.634c",
"Load.645",
"Load.646",
"Load.692",
"Load.675a",
"Load.675b",
"Load.675c",
"Load.611",
"Load.652",
"Load.670a",
"Load.670b",
"Load.670c"
]

lines = [
"Line.650632",
"Line.632670",
"Line.670671",
"Line.671680",
"Line.632633",
"Line.632645",
"Line.645646",
"Line.692675",
"Line.671684",
"Line.684611",
"Line.684652",
"Line.671692"
]

caps = [
"Capacitor.Cap1",
"Capacitor.Cap2"
]

transformers = [
"Transformer.Sub",
"Transformer.XFM1",
"Transformer.Reg1",
"Transformer.Reg2",
"Transformer.Reg3"
]

regulators = [
"regcontrol.Reg1",
"regcontrol.Reg2",
"regcontrol.Reg3"
]

#buses = dss.cktelement_read_bus_names()
#print(buses)

elements = lines

['671', '692']


show_before_and_after function is defined to show the voltages before and after the load is changed. This function is called in the next cell. It does so by generating 2 bar charts for the voltages before and after the load is changed. The function takes in the load name as a parameter and then uses the load name to get the bus name. The bus name is then used to get the voltages before and after the load is changed. The voltages are then plotted in a bar chart. The function returns the voltages before and after the load is changed.

In [6]:
def show_before_and_after(dict, item, parameter):
    fig, ax = plt.subplots(figsize=(6,6))
    for i in range(0, len(dict[item][parameter][0]), 2):
        ax.arrow(0, 0, dict[item][parameter][0][i], dict[item][parameter][0][i+1], head_width=10, head_length=10, fc='blue', ec='blue')
        ax.arrow(0, 0, dict[item][parameter][1][i], dict[item][parameter][1][i+1], head_width=10, head_length=10, fc='red', ec='red')
    ax.grid(True)
    #ax.legend(['Before'])
    ax.legend(['Before', 'After'])
    ax.set_xlabel('Real Axis (A)')
    ax.set_ylabel('Imaginary Axis (A)')
    title = 'Complex ' + parameter + ' Before ' + 'and After'
    ax.set_title(title)
    plt.show(block=False)

In [13]:
def show_voltages(col):
    buses_list = dss.circuit_all_node_names()
    bus_voltages = dss.circuit_all_bus_vmag_pu()
    fig = plt.figure(figsize = (75, 5))
    plt.bar(buses_list, bus_voltages, color =col)
    plt.xlabel("Nodes")
    plt.ylabel("Voltage (reference to ground in V)")
    plt.title("Voltage at each node in the circuit")
    plt.show(block=False)
    return

store_values function is called when some change has been made to a circuit, it stores the power and current values at all the pd and pc elements of the circuit.

In [17]:
def store_values():
    for load in loads:
        dss.circuit_set_active_element(load) 
        load_dict[load]["current"].append(dss.cktelement_currents())
        load_dict[load]["power"].append(dss.cktelement_powers())
    for line in lines:
        dss.circuit_set_active_element(line) 
        line_dict[line]["current"].append(dss.cktelement_currents())
        line_dict[line]["power"].append(dss.cktelement_powers())
    for cap in caps:
        dss.circuit_set_active_element(cap) 
        cap_dict[cap]["current"].append(dss.cktelement_currents())
        cap_dict[cap]["power"].append(dss.cktelement_powers())
    for transformer in transformers:
        dss.circuit_set_active_element(transformer) 
        transformer_dict[transformer]["current"].append(dss.cktelement_currents())
        transformer_dict[transformer]["power"].append(dss.cktelement_powers())
    for regulator in regulators:
        dss.circuit_set_active_element(regulator) 
        regulator_dict[regulator]["current"].append(dss.cktelement_currents())
        regulator_dict[regulator]["power"].append(dss.cktelement_powers())


change_loads function is used to change the load values of the circuit. The function takes in the name of the load, the phase, and the value to be changed to. The function then changes the load value and then solves the circuit. The function returns the voltages, powers, and currents at every node in the circuit.

In [31]:
def change_loads(load, value):
    dss.text(f"compile [{dss_file}]")
    dss.text(f"solve")
    dss.loads_write_name(load.split(".")[-1]) 
    dss.loads_write_kw(value) 
    dss.text(f"solve")
    store_values() 

In [19]:
def make_dicts():
    load_dict = {}
    for load in loads:
        load_dict[load] = {"power":[], "current":[]}
    line_dict = {}
    for line in lines:
        line_dict[line] = {"power":[], "current":[]}
    cap_dict = {}
    for cap in caps:
        cap_dict[cap] = {"power":[], "current":[]}
    transformer_dict = {}
    for transformer in transformers:
        transformer_dict[transformer] = {"power":[], "current":[]}
    regulator_dict = {}
    for regulator in regulators:
        regulator_dict[regulator] = {"power":[], "current":[]} 
    return load_dict, line_dict, cap_dict, transformer_dict, regulator_dict
    

In [27]:
def make_change(ld, val):
    dss.text(f"compile [{dss_file}]") 
    dss.text(f"solve")
    #show_voltages("blue")
    #store_values()
    change_loads(ld, val)

In [21]:
def bruteforce_placement():
    for _ in loads:
        make_change(_, 1155)

In [30]:
load_dict, line_dict, cap_dict, transformer_dict, regulator_dict = make_dicts()
make_change("Load.671", 755)
#store_values()
#print(load_dict["Load.671"]["current"])
#show_before_and_after(load_dict, "Load.671", "current")

In [33]:
def timeseries_load_parse(ld, arr, param):
    for i in range(len(arr)):
        change_loads(ld, arr[i])

In [34]:
load_dict, line_dict, cap_dict, transformer_dict, regulator_dict = make_dicts()
timeseries_load_parse("Load.671", [i*100 for i in range (1,11)], "kw")
print(load_dict["Load.671"]["current"])

[[13.254634274136464, -8.407410781481037, -13.90665816361276, -7.282714551270809, 0.6520238894762969, 15.690125332751846], [26.538306603100782, -16.91134210081399, -27.910734031372062, -14.542629637935795, 1.3724274282712798, 31.453971738749782], [39.688961831980336, -25.3891796569576, -41.82928355787042, -21.696504403086095, 2.140321725890084, 47.0856840600437], [52.9771477179861, -34.0464445356519, -55.9685740909518, -28.882853694333836, 2.9914263729656945, 62.92929822998573], [66.29533019547694, -42.80328630191946, -70.20841214305936, -36.0456797384077, 3.9130819475824374, 78.84896604032716], [79.6439649353333, -51.661251033480355, -84.55032671645093, -43.18459817289502, 4.906361781117624, 94.84584920637538], [93.02351955075255, -60.62192784084165, -98.9958882175181, -50.29921323423744, 5.972368666765551, 110.92114107507909], [106.43447389908425, -69.68695043152833, -113.54670986676541, -57.389117230258535, 7.1122359676811655, 127.07606766178688], [119.87784505842865, -78.8587669547

Some sample code to get all the bus names in the system.

In [35]:
buses_list = dss.circuit_all_node_names()
bus_voltages = dss.circuit_all_bus_vmag()